In [50]:
from functools import partial
import pandas as pd
import numpy as np

In [15]:
def organize_path(file_):
  return '/content/vinbigdata/train/' + file_ + '.jpg'

In [2]:
!cp /content/drive/MyDrive/datasets/vinbigdata-dataset.zip /content/
!unzip /content/vinbigdata-dataset.zip

In [16]:
vinbigdata_df = pd.read_csv('/content/vinbigdata/train.csv')
vinbigdata_df = vinbigdata_df.drop(['class_id', 'rad_id', 'x_min', 'y_min', 
                                    'x_max', 'y_max', 'width', 'height'], 
                                   axis = 1)

vinbigdata_df.rename(columns = {'image_id': 'Path'}, inplace = True)
path = list(vinbigdata_df['Path'])
path = list(map(organize_path, path))
vinbigdata_df['Path'] = path
vinbigdata_df['Dataset'] = ['VinBigData'] * len(vinbigdata_df)
vinbigdata_df['Sex'] = ['No Inf'] * len(vinbigdata_df)

In [ ]:
pneumothorax = vinbigdata_df[vinbigdata_df['class_name'] == 'Pneumothorax']
atelectasis = vinbigdata_df[vinbigdata_df['class_name'] == 'Atelectasis']
consolidation = vinbigdata_df[vinbigdata_df['class_name'] == 'Consolidation']
infiltration = vinbigdata_df[vinbigdata_df['class_name'] == 'Infiltration']
nodule_mass = vinbigdata_df[vinbigdata_df['class_name'] == 'Nodule/Mass']
thickening = vinbigdata_df[vinbigdata_df['class_name'] == 'Pleural thickening']
fibrosis = vinbigdata_df[vinbigdata_df['class_name'] == 'Pulmonary fibrosis']
opacity = vinbigdata_df[vinbigdata_df['class_name'] == 'Lung Opacity']
effusion = vinbigdata_df[vinbigdata_df['class_name'] == 'Pleural effusion']

dict_abnormalities = {'pneumothorax': pneumothorax, 'atelectasis': atelectasis,
                      'consolidation': consolidation, 'opacity': opacity,
                      'nodule_mass': nodule_mass, 'thickening': thickening,
                      'fibrosis': fibrosis, 'effusion': effusion, 
                      'infiltration': infiltration}

for keys, items in dict_abnormalities.items():
  dict_abnormalities[keys][keys] = [1] * len(items)
  columns_others = list(dict_abnormalities.keys())
  columns_others.remove(keys)
  zeros = np.zeros((len(items), 8))
  others_df = pd.DataFrame(data = zeros, columns = columns_others)
  dict_abnormalities[keys] = dict_abnormalities[keys].join(others_df)
  dict_abnormalities[keys] = dict_abnormalities[keys].fillna(0)
  columns = np.concatenate((['Dataset', 'Path', 'Sex', 'class_name'], 
                           list(dict_abnormalities.keys())))
  dict_abnormalities[keys] = dict_abnormalities[keys][columns]

In [65]:
vinbigdata_df = pd.concat((dict_abnormalities['pneumothorax'],
                           dict_abnormalities['atelectasis'],
                           dict_abnormalities['consolidation'],
                           dict_abnormalities['opacity'],
                           dict_abnormalities['nodule_mass'],
                           dict_abnormalities['thickening'],
                           dict_abnormalities['fibrosis'],
                           dict_abnormalities['effusion'],
                           dict_abnormalities['infiltration']))
vinbigdata_df = vinbigdata_df.drop(['class_name'], axis = 1)

In [68]:
columns = np.concatenate((['Dataset', 'Path', 'Sex'], 
                          list(dict_abnormalities.keys())))
vinbigdata_df.to_csv(path_or_buf = 'vinbigdata_df.csv', columns = columns)